In [2]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist_old,update_topology_old,spanning_tree_old

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic

In [3]:

#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123

requestedTime=1570041125



In [4]:
global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [ ]:
# RUN DEMO 9500 NODE SIMULATION
import Run9500NodeDemo as r
simulation_id = r.simulation.simulation_id
time.sleep(15)
print(simulation_id)
# Note: Restart and clear notebook kernel to re-run simulation

In [5]:
# Query for ACLineSegment, Transformerend, Switches, Breakers, Reclosers, Fuses, Sectionalisers,SynchronousMachine,TopologicalNode
topologyqueries.getallqueries(gapps,model_mrid)
Line_query=topologyqueries.Line_query
XfmrDict=topologyqueries.XfmrDict
XfmrKeys=topologyqueries.XfmrKeys
SwitchDict=topologyqueries.SwitchDict
SwitchKeys=topologyqueries.SwitchKeys
DG_query=topologyqueries.DG_query
Node_query=topologyqueries.Node_query

In [27]:
# Build Linknet Lists from ACLineSegment,Transformers,DGs and Nodes
[ConnNodeDict,TerminalsDict,TermList,NodeList]=linkedlist_old.build_linked_list(Line_query,XfmrDict,XfmrKeys,DG_query,Node_query)

# Stash a copy of base dictionary
BaseConnDict = json.dumps(ConnNodeDict)
BaseTermDict = json.dumps(TerminalsDict)

Processed  3909 line objects in  0.02371446399999999 seconds
Processed  1304 transformer objects in  0.008617179999999891 seconds
Processed  9 generator objects in  3.825499999998705e-05 seconds
Processed  4 missing nodes in  0.004265850999999987 seconds


In [8]:
# Process Switch Topology - run at each switch change - merges topology nodes across closed switches

[TerminalsDict,ConnNodeDict]=update_topology_old.topology_update(BaseConnDict,BaseTermDict,SwitchDict,SwitchKeys,TermList)

# Build Spanning Tree from Xfmrs & DGs


Processed  107 switch objects in  0.0002556310000000339 seconds


In [10]:

[Tree,TotalNodes]=spanning_tree_old.generate_spanning_tree(XfmrKeys,XfmrDict,ConnNodeDict,TerminalsDict,TermList,NodeList,DG_query)



Processed topology from substation transformer  hvmv69_11sub3_T1  with  2264  buses
Processed topology from substation transformer  hvmv69_11sub1_T1  with  1151  buses
Processed topology from substation transformer  hvmv69_11sub2_T1  with  1859  buses
Processed  3 topology trees containing  5274  buses in  0.21017483000000015 seconds
Processed  0  more islands containing  0  buses in  0.0012087640000000288 seconds
 
 


In [11]:
Tree.keys()

dict_keys(['_49F99DAD-8D58-49CE-979A-382CD0577D78', '_95DBCED6-E99E-4BB1-9229-899FB890050C', '_E701C38C-E8A0-4AFB-AC86-2C03158AE33D'])

In [24]:
ConnNodeDict[Tree['_95DBCED6-E99E-4BB1-9229-899FB890050C'][1]]

{'name': 'regxfmr_hvmv11sub1_lsb',
 'node': 5274,
 'list': 8617,
 'tpid': '_C849B8CA-5026-49C4-8FE0-DF8B68DEB6B8',
 'TransformerEnd.name': ['feeder_reg1c_T1'],
 'TransformerEnd.mRID': ['_4BFEC88B-7E8D-4BE2-A87C-8E7246BEC7A8'],
 'nomv': 0}

In [ ]:
#IF SIMULATION ID IS SPECIFIED, GET TOPOLOGY FOR REQUESTED TIME FOR SPECIFIED TIMESTAMP

equipment_dict = {}
measurement_dict = {}
meas_map={}

#Query for switch position measurement mRIDs
request = {"modelId": model_mrid,
           "requestType": "QUERY_OBJECT_MEASUREMENTS",
           "resultFormat": "JSON",
           "objectType": "LoadBreakSwitch"
           }

response = gapps.get_response(t.REQUEST_POWERGRID_DATA,request,timeout=15)

for measurement in response["data"]:
    if measurement["type"] == "Pos":
        measid = measurement["measid"]
        measurement_dict[measid] = measurement
        meas_map[measid] = measurement["eqid"]
        SwitchDict[measurement["eqid"]]["measid"] = measid

In [ ]:
topic = "goss.gridappsd.process.request.data.timeseries" # Specify Timeseries API GridAPPS-D topic

# Query for a particular set of measurments
message = {
    "queryMeasurement":"simulation",
    "queryFilter":{"simulation_id": str(simulation_id),
        "startTime": str(round(requestedTime-1)),
        "endTime": str(round(requestedTime+3)),
        "measurement_mrid": list(measurement_dict.keys())
                  },
        "responseFormat":"JSON" }


influx_response = gapps.get_response(t.TIMESERIES, message) # Pass API call

counter=0
while not influx_response['data'] and counter <5:
    influx_response = gapps.get_response(topic, message) # Pass API call
    print('Waiting 10 sec for data to be written to Timeseries Database')
    time.sleep(10)
    counter=counter+1
    if counter==5: print("No Timeseries data found. Returning default topology")

In [ ]:
for measurement in influx_response["data"]:
    measid = measurement["measurement_mrid"]
    time = measurement["time"]
    eqid = meas_map[measid]
    SwitchDict[eqid]["open"] = measurement['value']

In [ ]:
# Process Switch Topology - run at each switch change - merges topology nodes across closed switches

[TerminalsDict,ConnNodeDict]=update_topology_old.topology_update(BaseConnDict,BaseTermDict,SwitchDict,SwitchKeys,TermList)

# Build Spanning Tree from Xfmrs & DGs


In [ ]:
ConnNodeDict['_9B696653-55B1-47BA-9587-A7CB8A8774E7']

In [ ]:
TerminalsDict[TermList[4876]]

In [ ]:
TerminalsDict[TermList[4875]]

In [ ]:

[Tree,TotalNodes]=spanning_tree_old.generate_spanning_tree(XfmrKeys,XfmrDict,ConnNodeDict,TerminalsDict,TermList,NodeList,DG_query)

In [ ]:
SwitchDict

In [ ]:
ConnNodeDict['_03BBA6C7-58C4-4021-8CA4-9BED6D3825C9']

In [28]:
ConnNodeDict['_E118EFA9-CF00-4DC5-9D6C-5D7486C0F7E5']

{'name': 'm2001-ess1',
 'node': 5293,
 'list': 0,
 'tpid': '_49AF2E4A-6B77-47FD-B7D1-1BFD6F37FB06',
 'nomv': 0}

In [25]:
for i1 in range(1000):
    print(ConnNodeDict[Tree['_95DBCED6-E99E-4BB1-9229-899FB890050C'][i1]]['name'])

hvmv69sub1_hsb
regxfmr_hvmv11sub1_lsb
hvmv11sub1_lsb
hvmv_sub1_48332
d5710794-3_int
m1209817
e192860
l2801909
x2801909c
m1209814
sx2801909c
m1209811
m1209807
m1209805
l2823592
x2823592a
l2823592_cap
sx2823592a
r20185
d5513564-1_int
m1209800
e192201
l2839332
p829957
m1209797
l3160863
m1209791
x3160863c
l2730108
d5746703-1_int
p903354
m1209790
sx3160863c
x2730108c
l2786204
l2936270
m1209788
sx2730108c
x2786204c
m1209819
x2936270a
l3086078
m1209795
m1209787
sx2786204c
l2936216
sx2936270a
x3086078a
m4118755
l2936268
l2823611
x2936216c
226-23745
sx3086078a
m1209782
x2936268c
x2823611a
l3030204
sx2936216c
m1209822
l3217057
l3217056
sx2936268c
sx2823611a
x3030204a
m1186072
m1209823
l2992556
x3217057a
x3217056a
sx3030204a
m1186067
m1209824
x2992556c
sx3217057a
sx3217056a
m3032980
d6048634-1_int
m1209828
sx2992556c
m3032977
m3032981
l3253570
l2861157
m1166366
m1186065
x3253570c
l2937757
x2861157c
p901951
p830058
m1166368
d6504019-6_int
m1186064
p829977
sx3253570c
x2937757c
l3067448
sx2861157c
m